In [1]:
import os, sys
import sqlite3
import pandas as pd
from tqdm import tqdm
from pathlib import Path
from zipfile import ZipFile
from datatable import dt, f, by

# Descrição

Dados do Sistema de Informações Hospitalares do SUS (SIHSUS) reunidos pela Plataforma de Ciência de Dados aplicada à Saúde (PCDaS). Os dados foram obtidos junto ao DATASUS (Ministério da Saúde), tratados e enriquecidos seguindo uma metodologia própria de ETL (processo de extração, transformação e carga dos dados) da PCDaS, resultando em um dataset mensal com todos os registros de internações hospitalares contidos no SIHSUS à partir de janeiro de 2008.

A atualização dos dados é realizada pelo DATASUS e a PCDaS extrai os registros mensais referentes aos arquivos `RD_UF_ANO_MÊS.dbc` (Reduzidos de AIH) assim que disponibilizados. As colunas com nomes em MAIÚSCULO representam dados originais advindos do DATASUS e colunas com nomes iniciando em minúsculo representam dados resultantes de transformação ou enriquecimento.

# Funções Auxiliares

## Listar arquivos no ZIP

In [2]:
def get_zip_namelist(zip_path):
  files = ZipFile(zip_path).namelist()
  files = [f'{zip_path}/{f}'
    for f in files if 'dict' not in f]
  return files

## Abrir uma tabela

In [3]:
def read_table(table_path, columns):
  cols_order = list(columns.values())
  columns[...] = None
  df = dt.fread(table_path, columns=columns)
  return df

## Carregar tabelas do ZIP

In [4]:
def data_load(zip_path, cols, filter_func):
  files = get_zip_namelist(zip_path)
  dfs = list()
  for file_path in tqdm(files):
    df = read_table(file_path, cols)
    df = filter_func(df, cols)
    dfs.append(df)
  df = dt.rbind(*dfs)
  return df

## Agrupar por colunas

In [5]:
def group_cols(df_base, cols):
  df = df_base[:, cols]
  df['count'] = 1
  df = df[:, dt.sum(f.count), by(cols)]
  return df

# Filtragens básicas

## Filtro de Colunas

In [6]:
cols_sih = {
  # Infomações do procedimento:
  'ano_internacao': 'ano',
  'PROC_REA': 'parto_normal',
  'CNES': 'cnes',
  # Infomações da gestante:
  'def_idade_anos': 'idade',
  # Infomações de localidade do hospital de interção:
  'int_codigo_adotado': 'hosp_cod_municipio',
  'int_RSAUDCOD': 'hosp_regiao_saude',
  # Infomações de localidade da residência:
  'res_codigo_adotado': 'cod_municipio', # Armazena o código atribuído ao município de residência atualmente, tratando os casos em que múltiplos códigos tenham sido utilizados para um mesmo município ao longo do tempo
  'res_REGIAO': 'regiao', # Nome da região da unidade da federação de residência
  'res_SIGLA_UF': 'uf', # Sigla da unidade da federação de residência
  'res_CODIGO_UF': 'cod_uf', # Código da UF de residência
  'res_NOME_UF': 'nome_uf', # Nome da unidade da federação de residência
  'res_MUNNOME': 'municipio', # Nome do município de residência
  'res_MUNNOMEX': 'nome_municipio', # Nome do município de residência em maiúsculas e sem acentos
  'res_CAPITAL': 'capital', # Indica (S/N) se o município de residência é capital de UF
  'res_FRONTEIRA': 'fronteira', # Indica (S/N) se o município de residência faz parte da faixa de fronteira (conforme IBGE)
  'res_AMAZONIA': 'amazonia', # Indica (S/N) se o município de residência faz parte da Amazônia Legal (conforme IBGE)
  'res_MSAUDCOD': 'macroregiao_saude', # Código da Macrorregional de Saúde a que o Município de residência pertence
  'res_RSAUDCOD': 'regiao_saude', # Código da Regional de Saúde a que o Município de residência pertence
  'def_regiao_res': 'nome_regiao_saude', # Nome da Regional de Saúde a que o Município de residência pertence
  'res_CSAUDCOD': 'microregiao_saude', # Código da Microrregional de Saúde a que o Município de residência pertence
  'res_LATITUDE': 'latitude', # Latitude do município de residência
  'res_LONGITUDE': 'longitude', # Longitude do município de residência
  'res_ALTITUDE': 'altitude', # Altitude do município de residência
  'res_AREA': 'area', # Área do município de residência
}

## Filtro de Linhas

In [7]:
def filter_sih(df, cols):
  cols = list(cols.values())[:-1]
  cols = [c for c in cols if c in df.names]
  df_filtered = df[
    (
      (f['ano'] > 2009) & (f['ano'] < 2020)
    ) & (
      (f['idade'] > 9) & (f['idade'] < 50)
    ) & (
      (f['parto_normal'] == 310010039) | (f['parto_normal'] == 411010034)
    ) & (
      (f['cnes'] != None)
    ) & (
      (f['uf'] != 'DF')
    ) & (
      (f['regiao_saude'] != None)
    ) & (
      (f['hosp_regiao_saude'] != None)
    ) & (
      (f['cod_municipio'] != None)
    ), cols]
  return df_filtered

# Carregar dados

In [8]:
path_zip_sih = '/Volumes/SanDisk/Databases/SIH/SIHSUS.zip'
output_sih = path_zip_sih.replace('zip', 'jay')
# df_sih = data_load(path_zip_sih, cols_sih, filter_sih)
# df_sih.to_jay(output_sih) # to_jay | to_csv
df_sih = dt.fread(output_sih)
df_sih

,ano,parto_normal,cnes,idade,hosp_cod_municipio,hosp_regiao_saude,cod_municipio,regiao,uf,cod_uf,…,microregiao_saude,latitude,longitude,altitude,area
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,2010,310010039,2001020,23,120005,1202,120005,Norte,AC,12,…,12900,−10.941,−69.567,239,2875.91
1,2010,310010039,2000733,28,120040,1201,120040,Norte,AC,12,…,12900,−9.975,−67.81,153,9222.58
2,2010,310010039,2000733,38,120040,1201,120040,Norte,AC,12,…,12900,−9.975,−67.81,153,9222.58
3,2010,310010039,2000733,16,120040,1201,120040,Norte,AC,12,…,12900,−9.975,−67.81,153,9222.58
4,2010,310010039,2000733,26,120040,1201,120040,Norte,AC,12,…,12900,−9.975,−67.81,153,9222.58
5,2010,310010039,2000733,26,120040,1201,120040,Norte,AC,12,…,12900,−9.975,−67.81,153,9222.58
6,2010,310010039,2000733,27,120040,1201,120040,Norte,AC,12,…,12900,−9.975,−67.81,153,9222.58
7,2010,310010039,2001020,31,120005,1202,120005,Norte,AC,12,…,12900,−10.941,−69.567,239,2875.91
8,2010,310010039,2000733,30,120040,1201,120040,Norte,AC,12,…,12900,−9.975,−67.81,153,9222.58


100%|██████████| 4400/4400 [24:04<00:00,  3.05it/s]

Total: 28m 1.3s

# Selecionar

## Municípios

### Colunas da tabela

In [9]:
cols_mun = df_sih.names[6:]
cols_mun

('cod_municipio',
 'regiao',
 'uf',
 'cod_uf',
 'nome_uf',
 'municipio',
 'nome_municipio',
 'capital',
 'fronteira',
 'amazonia',
 'macroregiao_saude',
 'regiao_saude',
 'nome_regiao_saude',
 'microregiao_saude',
 'latitude',
 'longitude',
 'altitude',
 'area')

### Tabela resutante

In [10]:
df_mun = group_cols(df_sih, cols_mun)
df_mun = df_mun.sort(-f.count)
df_mun

,cod_municipio,regiao,uf,cod_uf,nome_uf,municipio,nome_municipio,capital,fronteira,amazonia,…,latitude,longitude,altitude,area,count
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,355030,Sudeste,SP,35,SAO PAULO,São Paulo,SAO PAULO,S,N,N,…,−23.548,−46.636,760,1522.99,806352
1,330455,Sudeste,RJ,33,RIO DE JANEIRO,Rio de Janeiro,RIO DE JANEIRO,S,N,N,…,−22.903,−43.208,2,1182.3,368216
2,130260,Norte,AM,13,AMAZONAS,Manaus,MANAUS,S,N,S,…,−3.102,−60.025,92,11401.1,297036
3,230440,Nordeste,CE,23,CEARA,Fortaleza,FORTALEZA,S,N,N,…,−3.717,−38.543,21,313.14,203370
4,292740,Nordeste,BA,29,BAHIA,Salvador,SALVADOR,S,N,N,…,−12.971,−38.511,8,706.799,169924
5,150140,Norte,PA,15,PARA,Belém,BELEM,S,N,S,…,−1.456,−48.504,10,1064.92,126918
6,310620,Sudeste,MG,31,MINAS GERAIS,Belo Horizonte,BELO HORIZONTE,S,N,N,…,−19.817,−43.956,858,330.954,125370
7,431490,Sul,RS,43,RIO GRANDE DO SUL,Porto Alegre,PORTO ALEGRE,S,N,N,…,−30.033,−51.23,3,496.827,105321
8,410690,Sul,PR,41,PARANA,Curitiba,CURITIBA,S,N,N,…,−25.428,−49.273,934,434.967,101337


### Exportando tabela

In [11]:
output_mun = f'{Path.home()}/Databases/GESTANTES/municipios.csv.gzip'
df_mun.to_csv(output_mun)

In [12]:
dt.fread(output_mun)

,cod_municipio,regiao,uf,cod_uf,nome_uf,municipio,nome_municipio,capital,fronteira,amazonia,…,latitude,longitude,altitude,area,count
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪
0,355030,Sudeste,SP,35,SAO PAULO,São Paulo,SAO PAULO,S,N,N,…,−23.548,−46.636,760,1522.99,806352
1,330455,Sudeste,RJ,33,RIO DE JANEIRO,Rio de Janeiro,RIO DE JANEIRO,S,N,N,…,−22.903,−43.208,2,1182.3,368216
2,130260,Norte,AM,13,AMAZONAS,Manaus,MANAUS,S,N,S,…,−3.102,−60.025,92,11401.1,297036
3,230440,Nordeste,CE,23,CEARA,Fortaleza,FORTALEZA,S,N,N,…,−3.717,−38.543,21,313.14,203370
4,292740,Nordeste,BA,29,BAHIA,Salvador,SALVADOR,S,N,N,…,−12.971,−38.511,8,706.799,169924
5,150140,Norte,PA,15,PARA,Belém,BELEM,S,N,S,…,−1.456,−48.504,10,1064.92,126918
6,310620,Sudeste,MG,31,MINAS GERAIS,Belo Horizonte,BELO HORIZONTE,S,N,N,…,−19.817,−43.956,858,330.954,125370
7,431490,Sul,RS,43,RIO GRANDE DO SUL,Porto Alegre,PORTO ALEGRE,S,N,N,…,−30.033,−51.23,3,496.827,105321
8,410690,Sul,PR,41,PARANA,Curitiba,CURITIBA,S,N,N,…,−25.428,−49.273,934,434.967,101337


## Procedimentos

### Colunas da tabela

In [13]:
cols_proc = [
  'ano',
  'parto_normal',
  'cnes',
  'hosp_cod_municipio',
  'hosp_regiao_saude',
  'cod_municipio',
  'regiao_saude',
  'idade'
]

### Tabela resutante

In [14]:
df_proc = group_cols(df_sih, cols_proc)
df_proc = df_proc.sort(f.ano, -f.count)
df_proc

,ano,parto_normal,cnes,hosp_cod_municipio,hosp_regiao_saude,cod_municipio,regiao_saude,idade,count
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪
0,2010,310010039,2077388,355030,3501,355030,3501,22,362
1,2010,310010039,2077388,355030,3501,355030,3501,20,333
2,2010,310010039,2077388,355030,3501,355030,3501,19,331
3,2010,310010039,2077388,355030,3501,355030,3501,24,327
4,2010,310010039,2077701,355030,3501,355030,3501,21,318
5,2010,310010039,2077388,355030,3501,355030,3501,21,303
6,2010,310010039,2077388,355030,3501,355030,3501,23,297
7,2010,310010039,2077701,355030,3501,355030,3501,20,296
8,2010,310010039,2077701,355030,3501,355030,3501,22,295


- with age: 3_355_691 rows × 9 columns
- without age: 461_276 rows × 8 columns

### Atualizando variável parto normal

In [15]:
df_proc[:, dt.update(parto_normal = f.parto_normal == 310010039)] # {310010039: 'normal', 411010034: 'cesariano'}
df_proc

,ano,parto_normal,cnes,hosp_cod_municipio,hosp_regiao_saude,cod_municipio,regiao_saude,idade,count
,▪▪▪▪,▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪
0,2010,1,2077388,355030,3501,355030,3501,22,362
1,2010,1,2077388,355030,3501,355030,3501,20,333
2,2010,1,2077388,355030,3501,355030,3501,19,331
3,2010,1,2077388,355030,3501,355030,3501,24,327
4,2010,1,2077701,355030,3501,355030,3501,21,318
5,2010,1,2077388,355030,3501,355030,3501,21,303
6,2010,1,2077388,355030,3501,355030,3501,23,297
7,2010,1,2077701,355030,3501,355030,3501,20,296
8,2010,1,2077701,355030,3501,355030,3501,22,295


### Exportando tabela

In [16]:
output_proc = f'{Path.home()}/Databases/GESTANTES/procedimentos.csv.gzip'
df_proc.to_csv(output_proc)

In [17]:
dt.fread(output_proc)

,ano,parto_normal,cnes,hosp_cod_municipio,hosp_regiao_saude,cod_municipio,regiao_saude,idade,count
,▪▪▪▪,▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,2010,1,2077388,355030,3501,355030,3501,22,362
1,2010,1,2077388,355030,3501,355030,3501,20,333
2,2010,1,2077388,355030,3501,355030,3501,19,331
3,2010,1,2077388,355030,3501,355030,3501,24,327
4,2010,1,2077701,355030,3501,355030,3501,21,318
5,2010,1,2077388,355030,3501,355030,3501,21,303
6,2010,1,2077388,355030,3501,355030,3501,23,297
7,2010,1,2077701,355030,3501,355030,3501,20,296
8,2010,1,2077701,355030,3501,355030,3501,22,295
